In [1]:
from __future__ import division, print_function, absolute_import
import csv
import pandas
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)
import os
import xml.etree.ElementTree as etree
import cobra
import numpy as np
from itertools import chain
from cobra.util import solver as sutil
from cobra.core.solution import get_solution
from optlang.symbolics import add, Zero
import pandas as pd
import os
from os.path import join
import matplotlib.pyplot as plt
from cobra.medium import minimal_medium
from cobra.flux_analysis import production_envelope
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import flux_variability_analysis
import matplotlib.pyplot as plt
from cobra.io import load_json_model, save_json_model, load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model
import math

In [2]:
model_rs = read_sbml_model('/Users/subasrees/Desktop/Core_Model_Simulations/beta_day_RS_DM_r.xml')
core_model=model_rs

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-16


In [3]:
## get the metabolites from the biomass reaction
a=[]
a=core_model.reactions.get_by_id('AraCore_Biomass_tx').reactants


In [4]:
## print the model summary
core_model.reactions.get_by_id('AraCore_Biomass_tx').summary()


In [5]:
## get the biomass coefficients
b=[]
a=core_model.reactions.get_by_id('AraCore_Biomass_tx').metabolites
for key in a:
    b.append(a[key])

In [6]:
## calculate the molecular weight of the biomass precursors
c=[]
for i in a:
    #print(i)
    c.append(core_model.metabolites.get_by_id(i.id).formula_weight)

In [7]:
## Create a dataframe of the stoichiometric coefficients, molecular weight 
df=pd.DataFrame(b,dtype='float',index=list(a))
df.columns=['s']
df['for_weight']=c
## calculate the weight from molecular weight and coefficients
df['weight']=-1*df['for_weight']*df['s']

In [8]:
## rescale the coefficients
biomass_factor=df['weight'].sum()
df['new_coeff']=df['s']/biomass_factor
met_list=df.index
coef=df['new_coeff']

In [9]:
core_model.reactions.remove('AraCore_Biomass_tx')
## 
reaction = Reaction('AraCore_Biomass_tx')
reaction.name = 'Arabidopsis biomass'
reaction.lower_bound = 0.  
reaction.upper_bound = 1000.  
#Add reaction to the model
core_model.add_reactions([reaction])

In [10]:
## add the rescaled biomass coefficients into the biomass reaction
for i in range(len(coef)):
    core_model.reactions.get_by_id('AraCore_Biomass_tx').add_metabolites({met_list[i]:coef[i]})

/var/folders/yl/47vnp0617dn9zfv19_vrxwjh0000gn/T/ipykernel_82167/2478426878.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  core_model.reactions.get_by_id('AraCore_Biomass_tx').add_metabolites({met_list[i]:coef[i]})


In [11]:
##Constraints
rubisco = core_model.problem.Constraint(3 * core_model.reactions.get_by_id("RXN_961_p").flux_expression - core_model.reactions.get_by_id("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p").flux_expression,lb=0, ub=0,)
core_model.add_cons_vars([rubisco])

atp = core_model.problem.Constraint((0.0049*core_model.reactions.get_by_id("Photon_tx").flux_expression+2.7851)-core_model.reactions.get_by_id("ATPase_tx").flux_expression, lb=0, ub=0)
core_model.add_cons_vars(atp)

atp_nadph_03 = core_model.problem.Constraint(3 * (core_model.reactions.get_by_id("NADPHoxm_tx").flux_expression + core_model.reactions.get_by_id("NADPHoxc_tx").flux_expression + core_model.reactions.get_by_id("NADPHoxp_tx").flux_expression) - core_model.reactions.get_by_id("ATPase_tx").flux_expression, lb=0, ub=0)
core_model.add_cons_vars(atp_nadph_03)

In [12]:
core_model.reactions.get_by_id('AraCore_Biomass_tx').summary()

In [13]:
#write_sbml_model(core_model,'beta_day_RS_DM_new.xml')